# test-model

Test model accuracy against a couple images it never saw during training.

In [1]:
from PIL import Image
from typing import List

import pandas as pd

import torch
import torch.nn as nn
import torchvision.models as models

from torchvision.transforms import v2

In [2]:
IMG_SIZE = 256

BAND_COLORS = [
    'silver', 'white', 'blue', 'grey', 'violet', 'green', 
    'yellow', 'orange', 'red', 'gold', 'black', 'brown'
]
MAX_BANDS = 4
CLASSES = MAX_BANDS * len(BAND_COLORS)

In [3]:
# load neural net, must match saved model

# model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
# model.fc = nn.Sequential(
#     nn.Linear(in_features=model.fc.in_features, out_features=CLASSES),
#     # nn.Sigmoid(),
# )

In [4]:
# model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
# model.fc = nn.Sequential(
#     nn.Linear(in_features=model.fc.in_features, out_features=CLASSES),
#     # nn.Sigmoid(),
# )

In [5]:
# model = models.efficientnet_v2_s(weights=models.EfficientNet_V2_S_Weights.DEFAULT)
# model.classifier[1] = nn.Linear(in_features=model.classifier[1].in_features, out_features=CLASSES)
# # model.classifier.append(nn.Sigmoid())

In [6]:
model = models.efficientnet_v2_m(weights=models.EfficientNet_V2_M_Weights.DEFAULT)
model.classifier[1] = nn.Linear(in_features=model.classifier[1].in_features, out_features=CLASSES)
# model.classifier.append(nn.Sigmoid())

In [7]:
model.load_state_dict(torch.load('../what-the-ohm-4B.pth'))
# model.load_state_dict(torch.load('../what-the-ohm.pth'))

<All keys matched successfully>

In [8]:
# generate list of classes
def gen_classes() -> List[str]:
    classes = []
    for i in range(1, MAX_BANDS + 1):
        for color in BAND_COLORS:
            classes.append(f'band_{i}_{color}')
    return classes

# use model to predict bands in image
def predict(model: nn.Module, img_path: str):
    preprocess = v2.Compose([
        v2.Resize(IMG_SIZE),
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ])

    img = Image.open(img_path)
    img_tensor = preprocess(img)
    batch = img_tensor.unsqueeze(0) # add batch dimension

    model.eval()

    with torch.no_grad():
        logits = model(batch).squeeze(0) # remove batch dimension
        # print(logits)
        class_probs = torch.sigmoid(logits)
    return class_probs

# convert band probabilities to classes
def probs_to_classes(probs: torch.Tensor, threshold: float = 0.5) -> List[str]:
    classes = gen_classes()

    for i,c in enumerate(classes):
        if i > 0 and i % len(BAND_COLORS) == 0:
            print()
        p = probs[i] * 100.0
        print(f'{c} -> {p:.2f}%')

    pred_indices = (probs > threshold).nonzero()[:, 0]
    return [classes[i] for i in pred_indices]

# test model and print results
def test_model(model: nn.Module, img_path: str, actual_classes: List[str]):
    probs = predict(model, img_path)
    pred_classes = probs_to_classes(probs)

    print('\npredicted classes:', pred_classes)
    print('actual classes:   ', actual_classes)


## Test 1

4 band, 1M ohm, 5% tolerance

<img src="test-data/4B-1M-T5.jpg" alt="test-data/4B-1M-T5.jpg" width="256"/>

In [9]:
test_model(model, 'test-data/4B-1M-T5.jpg', actual_classes=['band_1_brown', 'band_2_black', 'band_3_green', 'band_4_gold'])

band_1_silver -> 0.07%
band_1_white -> 0.04%
band_1_blue -> 0.03%
band_1_grey -> 0.09%
band_1_violet -> 0.06%
band_1_green -> 0.08%
band_1_yellow -> 0.22%
band_1_orange -> 0.12%
band_1_red -> 0.89%
band_1_gold -> 0.05%
band_1_black -> 0.06%
band_1_brown -> 99.37%

band_2_silver -> 0.05%
band_2_white -> 0.13%
band_2_blue -> 0.08%
band_2_grey -> 0.21%
band_2_violet -> 0.09%
band_2_green -> 3.85%
band_2_yellow -> 0.07%
band_2_orange -> 0.06%
band_2_red -> 1.32%
band_2_gold -> 0.04%
band_2_black -> 92.60%
band_2_brown -> 0.15%

band_3_silver -> 0.02%
band_3_white -> 0.06%
band_3_blue -> 0.09%
band_3_grey -> 0.04%
band_3_violet -> 0.06%
band_3_green -> 99.66%
band_3_yellow -> 1.24%
band_3_orange -> 0.63%
band_3_red -> 0.15%
band_3_gold -> 3.82%
band_3_black -> 0.10%
band_3_brown -> 0.21%

band_4_silver -> 0.06%
band_4_white -> 0.11%
band_4_blue -> 0.09%
band_4_grey -> 0.05%
band_4_violet -> 0.04%
band_4_green -> 0.05%
band_4_yellow -> 0.03%
band_4_orange -> 0.05%
band_4_red -> 0.06%
band_4_

## Test 2

4 band, 4.7K ohm, 5% tolerance

<img src="test-data/4B-4K7-T5.jpg" alt="test-data/4B-4K7-T5.jpg" width="256"/>

In [10]:
test_model(model, 'test-data/4B-4K7-T5.jpg', actual_classes=['band_1_yellow', 'band_2_violet', 'band_3_red', 'band_4_gold'])

band_1_silver -> 0.04%
band_1_white -> 0.06%
band_1_blue -> 0.05%
band_1_grey -> 0.08%
band_1_violet -> 0.04%
band_1_green -> 0.01%
band_1_yellow -> 98.50%
band_1_orange -> 0.14%
band_1_red -> 21.74%
band_1_gold -> 0.05%
band_1_black -> 0.05%
band_1_brown -> 0.03%

band_2_silver -> 0.05%
band_2_white -> 0.47%
band_2_blue -> 0.02%
band_2_grey -> 0.01%
band_2_violet -> 99.99%
band_2_green -> 0.06%
band_2_yellow -> 0.03%
band_2_orange -> 0.14%
band_2_red -> 0.17%
band_2_gold -> 0.06%
band_2_black -> 0.06%
band_2_brown -> 0.03%

band_3_silver -> 0.04%
band_3_white -> 0.04%
band_3_blue -> 0.04%
band_3_grey -> 0.03%
band_3_violet -> 0.07%
band_3_green -> 0.12%
band_3_yellow -> 0.44%
band_3_orange -> 51.17%
band_3_red -> 0.62%
band_3_gold -> 2.12%
band_3_black -> 2.62%
band_3_brown -> 1.38%

band_4_silver -> 0.03%
band_4_white -> 0.04%
band_4_blue -> 0.03%
band_4_grey -> 0.05%
band_4_violet -> 0.01%
band_4_green -> 0.04%
band_4_yellow -> 0.03%
band_4_orange -> 0.04%
band_4_red -> 0.02%
band_4

## Test 3

4 band, 47K ohm, 5% tolerance

<img src="test-data/4B-47K-T5.jpg" alt="test-data/4B-47K-T5.jpg" width="256"/>

In [11]:
test_model(model, 'test-data/4B-47K-T5.jpg', actual_classes=['band_1_yellow', 'band_2_violet', 'band_3_orange', 'band_4_gold'])

band_1_silver -> 0.08%
band_1_white -> 0.08%
band_1_blue -> 0.23%
band_1_grey -> 0.06%
band_1_violet -> 0.06%
band_1_green -> 0.14%
band_1_yellow -> 97.44%
band_1_orange -> 1.09%
band_1_red -> 0.74%
band_1_gold -> 0.09%
band_1_black -> 0.06%
band_1_brown -> 0.17%

band_2_silver -> 0.11%
band_2_white -> 1.64%
band_2_blue -> 0.24%
band_2_grey -> 0.09%
band_2_violet -> 99.49%
band_2_green -> 0.40%
band_2_yellow -> 0.06%
band_2_orange -> 0.26%
band_2_red -> 0.09%
band_2_gold -> 0.06%
band_2_black -> 0.09%
band_2_brown -> 0.12%

band_3_silver -> 0.07%
band_3_white -> 0.07%
band_3_blue -> 0.08%
band_3_grey -> 0.07%
band_3_violet -> 0.13%
band_3_green -> 0.17%
band_3_yellow -> 0.13%
band_3_orange -> 99.81%
band_3_red -> 0.18%
band_3_gold -> 0.89%
band_3_black -> 0.53%
band_3_brown -> 0.18%

band_4_silver -> 0.05%
band_4_white -> 0.06%
band_4_blue -> 0.07%
band_4_grey -> 0.09%
band_4_violet -> 0.02%
band_4_green -> 0.08%
band_4_yellow -> 0.08%
band_4_orange -> 0.08%
band_4_red -> 0.03%
band_4_

## Test 4

4 band, 220 ohm, 5% tolerance

<img src="test-data/4B-220R-T5.jpg" alt="test-data/4B-220R-T5.jpg" width="256"/>

In [12]:
test_model(model, 'test-data/4B-220R-T5.jpg', actual_classes=['band_1_red', 'band_2_red', 'band_3_brown', 'band_4_gold'])

band_1_silver -> 0.27%
band_1_white -> 0.13%
band_1_blue -> 0.33%
band_1_grey -> 8.66%
band_1_violet -> 0.11%
band_1_green -> 0.80%
band_1_yellow -> 0.13%
band_1_orange -> 0.46%
band_1_red -> 98.27%
band_1_gold -> 0.24%
band_1_black -> 0.29%
band_1_brown -> 0.62%

band_2_silver -> 0.13%
band_2_white -> 0.23%
band_2_blue -> 0.72%
band_2_grey -> 0.25%
band_2_violet -> 1.03%
band_2_green -> 1.30%
band_2_yellow -> 0.22%
band_2_orange -> 0.96%
band_2_red -> 99.21%
band_2_gold -> 0.23%
band_2_black -> 0.19%
band_2_brown -> 0.70%

band_3_silver -> 0.19%
band_3_white -> 0.25%
band_3_blue -> 0.17%
band_3_grey -> 0.13%
band_3_violet -> 0.27%
band_3_green -> 6.20%
band_3_yellow -> 0.17%
band_3_orange -> 0.49%
band_3_red -> 0.48%
band_3_gold -> 0.47%
band_3_black -> 4.35%
band_3_brown -> 99.57%

band_4_silver -> 0.23%
band_4_white -> 0.17%
band_4_blue -> 0.09%
band_4_grey -> 0.14%
band_4_violet -> 0.22%
band_4_green -> 0.30%
band_4_yellow -> 0.17%
band_4_orange -> 0.23%
band_4_red -> 0.32%
band_4_

## Test 5

4 band, 22K ohm, 5% tolerance

<img src="test-data/4B-22K-T5.jpg" alt="test-data/4B-22K-T5.jpg" width="256"/>

In [13]:
test_model(model, 'test-data/4B-22K-T5.jpg', actual_classes=['band_1_red', 'band_2_red', 'band_3_orange', 'band_4_gold'])

band_1_silver -> 0.06%
band_1_white -> 0.03%
band_1_blue -> 0.19%
band_1_grey -> 0.18%
band_1_violet -> 0.05%
band_1_green -> 0.08%
band_1_yellow -> 0.02%
band_1_orange -> 30.45%
band_1_red -> 97.15%
band_1_gold -> 0.05%
band_1_black -> 0.04%
band_1_brown -> 0.12%

band_2_silver -> 0.04%
band_2_white -> 0.99%
band_2_blue -> 0.06%
band_2_grey -> 0.09%
band_2_violet -> 0.09%
band_2_green -> 0.04%
band_2_yellow -> 0.05%
band_2_orange -> 11.52%
band_2_red -> 98.53%
band_2_gold -> 0.04%
band_2_black -> 0.01%
band_2_brown -> 0.04%

band_3_silver -> 0.06%
band_3_white -> 0.03%
band_3_blue -> 0.03%
band_3_grey -> 0.06%
band_3_violet -> 0.07%
band_3_green -> 0.58%
band_3_yellow -> 0.27%
band_3_orange -> 99.48%
band_3_red -> 0.13%
band_3_gold -> 0.24%
band_3_black -> 2.30%
band_3_brown -> 0.08%

band_4_silver -> 0.03%
band_4_white -> 0.02%
band_4_blue -> 0.04%
band_4_grey -> 0.04%
band_4_violet -> 0.03%
band_4_green -> 0.05%
band_4_yellow -> 0.03%
band_4_orange -> 0.04%
band_4_red -> 0.04%
band_